# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [271]:
import pandas as pd

school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [272]:
school_count = school_data_complete["school_name"].nunique()
school_count

15

In [273]:
student_count = school_data_complete["student_name"].count()
student_count

39170

In [274]:
total_budget = school_data_complete["budget"].unique()
total_budget.sum()

24649428

In [275]:
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [276]:
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [277]:
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [278]:
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [279]:
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [280]:
district_summary = pd.DataFrame({'School Count': school_count, 
'Total Students': student_count, 
'Total Budget': total_budget, 
'Avg Math Score': average_math_score, 
'Avg Reading Score': average_reading_score, 
'% Passing Math': passing_math_percentage,
'% Passing Reading': passing_reading_percentage, 
'% Overall Passing': overall_passing_rate})

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary.head(1)

,School Count,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$1,910,635.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [281]:
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [282]:
per_school_counts = school_data.set_index(["school_name"])["size"]
per_school_counts

school_name
Huang High School        2917
Figueroa High School     2949
Shelton High School      1761
Hernandez High School    4635
Griffin High School      1468
Wilson High School       2283
Cabrera High School      1858
Bailey High School       4976
Holden High School        427
Pena High School          962
Wright High School       1800
Rodriguez High School    3999
Johnson High School      4761
Ford High School         2739
Thomas High School       1635
Name: size, dtype: int64

In [283]:
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts
per_school_capita

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [284]:
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

In [285]:
school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)].nunique()["school_name"]
school_passing_math

15

In [286]:
school_passing_reading = school_data_complete[(school_data_complete["reading_score"]>=70)].nunique()["school_name"]
school_passing_reading

15

In [287]:
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]


In [288]:
per_school_passing_math = school_passing_math / per_school_counts * 100
per_school_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["reading_score"] / per_school_counts * 100

In [289]:
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Avg Math Score": per_school_math,
    "Avg Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": overall_passing_rate
})

per_school_summary.index.names = ["School Name"]

per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,0.301447,0.301447,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,0.807320,0.807320,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,0.508647,0.508647,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,0.547645,0.547645,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1.021798,1.021798,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,0.323625,0.323625,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,3.512881,3.512881,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,0.514227,0.514227,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,0.315060,0.315060,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [290]:
highest_df = per_school_summary.sort_values(by="% Overall Passing", ascending=False)
highest_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,0.807320,0.807320,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,0.917431,0.917431,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1.021798,1.021798,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,0.657030,0.657030,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,1.559252,1.559252,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [291]:
bottom_df = per_school_summary.sort_values(by="% Overall Passing", ascending=True)
bottom_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,0.375094,0.375094,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,0.508647,0.508647,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,0.514227,0.514227,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,0.323625,0.323625,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,0.315060,0.315060,53.539172


## Math Scores by Grade

In [292]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores.mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

math_scores_by_grade = pd.DataFrame({
    "9th Grade Math Scores": ninth_grade_math_scores,
    "10th Grade Math Scores": tenth_grader_math_scores,
    "11th Grade Math Scores": eleventh_grader_math_scores,
    "12th Grade Math Scores": twelfth_grader_math_scores
})

math_scores_by_grade.index.name = None

math_scores_by_grade

,9th Grade Math Scores,10th Grade Math Scores,11th Grade Math Scores,12th Grade Math Scores
Bailey High School,77.083676,76.996772,80.575873,76.492218
Cabrera High School,83.094697,83.154506,80.575873,83.277487
Figueroa High School,76.403037,76.539974,80.575873,77.151369
Ford High School,77.361345,77.672316,80.575873,76.179963
Griffin High School,82.044010,84.229064,80.575873,83.356164
Hernandez High School,77.438495,77.337408,80.575873,77.186567
Holden High School,83.787402,83.429825,80.575873,82.855422
Huang High School,77.027251,75.908735,80.575873,77.225641
Johnson High School,77.187857,76.691117,80.575873,76.863248
Pena High School,83.625455,83.372000,80.575873,84.121547


## Reading Score by Grade 

In [293]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores.mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

reading_scores_by_grade = pd.DataFrame({
    "9th Grade Reading Scores": ninth_grade_reading_scores,
    "10th Grade Reading Scores": tenth_grader_reading_scores,
    "11th Grade Reading Scores": eleventh_grader_reading_scores,
    "12th Grade Reading Scores": twelfth_grader_reading_scores
})

reading_scores_by_grade = reading_scores_by_grade[["9th Grade Reading Scores", "10th Grade Reading Scores", 
"11th Grade Reading Scores", "12th Grade Reading Scores"]]
reading_scores_by_grade.index.name = None

reading_scores_by_grade

,9th Grade Reading Scores,10th Grade Reading Scores,11th Grade Reading Scores,12th Grade Reading Scores
Bailey High School,81.303155,80.907183,82.559485,80.912451
Cabrera High School,83.676136,84.253219,82.559485,84.287958
Figueroa High School,81.198598,81.408912,82.559485,81.384863
Ford High School,80.632653,81.262712,82.559485,80.662338
Griffin High School,83.369193,83.706897,82.559485,84.013699
Hernandez High School,80.866860,80.660147,82.559485,80.857143
Holden High School,83.677165,83.324561,82.559485,84.698795
Huang High School,81.290284,81.512386,82.559485,80.305983
Johnson High School,81.260714,80.773431,82.559485,81.227564
Pena High School,83.807273,83.612000,82.559485,84.591160


## Scores by School Spending

In [294]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [295]:
school_spending_df = per_school_summary.copy()

In [296]:
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, 
    spending_bins, labels=labels, include_lowest=True)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
School Name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,0.301447,0.301447,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,0.807320,0.807320,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,0.508647,0.508647,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,0.547645,0.547645,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1.021798,1.021798,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,0.323625,0.323625,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,3.512881,3.512881,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,0.514227,0.514227,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,0.315060,0.315060,53.539172,$645-680


In [297]:
 spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Avg Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Avg Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [298]:
spending_summary = pd.DataFrame({
    "Avg Math Score": spending_math_scores,
    "Avg Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

spending_summary

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,1.452641,1.452641,90.369459
$585-630,81.899826,83.155286,0.933571,0.933571,81.418596
$630-645,78.518855,81.624473,0.587204,0.587204,62.857656
$645-680,76.997210,81.027843,0.384304,0.384304,53.526855


## Scores by School Size

In [299]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [300]:
per_school_summary["School Size"] = pd.cut(per_school_counts, size_bins, labels=labels, include_lowest=True)
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
School Name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,0.301447,0.301447,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,0.807320,0.807320,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,0.508647,0.508647,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,0.547645,0.547645,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,1.021798,1.021798,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,0.323625,0.323625,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,3.512881,3.512881,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,0.514227,0.514227,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,0.315060,0.315060,53.539172,Large (2000-5000)


In [301]:
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Avg Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Avg Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [302]:
size_summary = pd.DataFrame({
    "Avg Math Score": size_math_scores,
    "Avg Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})

size_summary

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,2.536066,2.536066,89.883853
Medium (1000-2000),83.374684,83.864438,0.886334,0.886334,90.621535
Large (2000-5000),77.746417,81.344493,0.442847,0.442847,58.286003


## Scores by School Type

In [303]:
type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Avg Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Avg Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()["% Overall Passing"]

In [304]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Avg Math Score": type_math_scores,
    "Avg Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": type_overall_passing
})

# Display results
type_summary

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,1.270104,1.270104,90.432244
District,76.956733,80.966636,0.412249,0.412249,53.672208
